In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

In [2]:
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


## Weighted Scoring Model for finding the final score

In [7]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
    
    numeric_cols = [
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    
    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [5]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,40.0,5.50,8.300000,44.0,78.4,0.041176,0.033582,0.046889,0.042889,0.651948
1,TCS.NS,197.0,61.00,52.140000,131.0,6.0,0.225882,0.447761,0.357878,0.141084,0.338528
2,HDFCBANK.NS,115.0,11.00,25.150000,67.0,-2.2,0.129412,0.074627,0.166418,0.068849,0.303030
3,INFY.NS,420.0,63.00,65.900004,216.0,5.6,0.488235,0.462687,0.455487,0.237020,0.336797
4,ICICIBANK.NS,79.0,11.00,13.600000,99.0,14.6,0.087059,0.074627,0.084486,0.104966,0.375758
5,HINDUNILVR.NS,181.0,48.00,93.600000,154.0,5.6,0.207059,0.350746,0.651983,0.167043,0.336797
6,SBIN.NS,198.0,15.90,17.860001,139.0,9.7,0.227059,0.111194,0.114705,0.150113,0.354545
7,BAJFINANCE.NS,50.0,4.40,15.680000,24.0,19.7,0.052941,0.025373,0.099241,0.020316,0.397835
8,BHARTIARTL.NS,85.0,16.00,13.570000,80.0,41.8,0.094118,0.111940,0.084273,0.083521,0.493506
9,ITC.NS,392.0,15.70,90.029997,406.0,2.7,0.455294,0.109701,0.626658,0.451467,0.324242


In [10]:
value_cols = [
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]

for col in value_cols:
    dividend_df[col] = dividend_df[col].fillna(dividend_df[col].mean())
    
dividend_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 30 to 10
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Ticker                                   50 non-null     object 
 1   Dividend Yield(%)                        50 non-null     float64
 2   Dividend Rate                            50 non-null     float64
 3   Payout Ratio(%)                          50 non-null     float64
 4   5 Year Avg Dividend Yield(%)             50 non-null     float64
 5   Earning Growth(%)                        50 non-null     float64
 6   Dividend Yield(%) Normalised             48 non-null     float64
 7   Dividend Rate Normalised                 48 non-null     float64
 8   Payout Ratio(%) Normalised               46 non-null     float64
 9   5 Year Avg Dividend Yield(%) Normalised  46 non-null     float64
 10  Earning Growth(%) Normalised             49 non-null    

In [12]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
30,COALINDIA.NS,855.000000,32.000000,49.000000,892.000000,-20.200000,1.000000,0.231343,0.335603,1.000000,0.225108,0.635900
42,HEROMOTOCO.NS,255.000000,130.000000,55.630000,310.000000,63.100000,0.294118,0.962687,0.382635,0.343115,0.585714,0.484494
28,TECHM.NS,301.000000,45.000000,142.660000,216.000000,34.000000,0.348235,0.328358,1.000000,0.237020,0.459740,0.463520
20,ONGC.NS,632.000000,14.750000,47.100000,517.000000,-1.400000,0.737647,0.102612,0.322125,0.576749,0.306494,0.452241
13,HCLTECH.NS,372.000000,54.000000,86.300004,313.000000,-7.700000,0.431765,0.395522,0.600199,0.346501,0.279221,0.425896
3,INFY.NS,420.000000,63.000000,65.900004,216.000000,5.600000,0.488235,0.462687,0.455487,0.237020,0.336797,0.411189
9,ITC.NS,392.000000,15.700000,90.029997,406.000000,2.700000,0.455294,0.109701,0.626658,0.451467,0.324242,0.406578
22,POWERGRID.NS,428.000000,11.750000,65.639997,504.000000,-2.500000,0.497647,0.080224,0.453643,0.562077,0.301732,0.398656
31,BPCL.NS,322.000000,10.000000,38.210000,557.000000,140.800000,0.372941,0.067164,0.259062,0.621896,0.922078,0.393715
45,IOC.NS,217.000000,3.000000,57.100000,725.000000,93.400000,0.249412,0.014925,0.393062,0.811512,0.716883,0.390412


In [14]:
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
30,COALINDIA.NS,855.0,32.00,49.000000,892.0,-20.2,1.000000,0.231343,0.335603,1.000000,0.225108,0.635900
42,HEROMOTOCO.NS,255.0,130.00,55.630000,310.0,63.1,0.294118,0.962687,0.382635,0.343115,0.585714,0.484494
28,TECHM.NS,301.0,45.00,142.660000,216.0,34.0,0.348235,0.328358,1.000000,0.237020,0.459740,0.463520
20,ONGC.NS,632.0,14.75,47.100000,517.0,-1.4,0.737647,0.102612,0.322125,0.576749,0.306494,0.452241
13,HCLTECH.NS,372.0,54.00,86.300004,313.0,-7.7,0.431765,0.395522,0.600199,0.346501,0.279221,0.425896
3,INFY.NS,420.0,63.00,65.900004,216.0,5.6,0.488235,0.462687,0.455487,0.237020,0.336797,0.411189
9,ITC.NS,392.0,15.70,90.029997,406.0,2.7,0.455294,0.109701,0.626658,0.451467,0.324242,0.406578
22,POWERGRID.NS,428.0,11.75,65.639997,504.0,-2.5,0.497647,0.080224,0.453643,0.562077,0.301732,0.398656
31,BPCL.NS,322.0,10.00,38.210000,557.0,140.8,0.372941,0.067164,0.259062,0.621896,0.922078,0.393715
45,IOC.NS,217.0,3.00,57.100000,725.0,93.4,0.249412,0.014925,0.393062,0.811512,0.716883,0.390412
